In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import datetime
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, Input, LSTM
from keras.models import Sequential, Model,load_model
from keras.layers import Activation, Dense, Dropout, Embedding, Flatten, Input, Convolution1D, MaxPooling1D, GlobalMaxPooling1D
import numpy as np
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from torch import optim
from sklearn.preprocessing import MinMaxScaler
import torch.nn as nn
import datetime

Using TensorFlow backend.


In [2]:
data=pd.read_csv("agg_final_data.csv")

In [3]:
scaler = MinMaxScaler()
data["unit_sales"]=scaler.fit_transform(np.array(data.unit_sales).reshape(-1,1))

In [4]:
store_data=pd.read_csv("stores_agg.csv")

In [5]:
item_data=pd.read_csv("items_agg.csv")

### Stores_Model

In [7]:
import re

def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]

In [8]:
m=[i for i in store_data.columns if "store" in i]

In [9]:
m.sort(key=natural_keys)

In [10]:
a=np.array(store_data[m])

In [11]:
store_data["store_nbr"]=[np.where(r==1)[0][0]+1 for r in a]

In [12]:
X1 = torch.tensor(store_data[['store_nbr_1', 'store_nbr_10', 'store_nbr_11', 'store_nbr_12',
       'store_nbr_13', 'store_nbr_14', 'store_nbr_15', 'store_nbr_16',
       'store_nbr_17', 'store_nbr_18', 'store_nbr_19', 'store_nbr_2',
       'store_nbr_20', 'store_nbr_21', 'store_nbr_22', 'store_nbr_23',
       'store_nbr_24', 'store_nbr_25', 'store_nbr_26', 'store_nbr_27',
       'store_nbr_28', 'store_nbr_29', 'store_nbr_3', 'store_nbr_30',
       'store_nbr_31', 'store_nbr_32', 'store_nbr_33', 'store_nbr_34',
       'store_nbr_35', 'store_nbr_36', 'store_nbr_37', 'store_nbr_38',
       'store_nbr_39', 'store_nbr_4', 'store_nbr_40', 'store_nbr_41',
       'store_nbr_42', 'store_nbr_43', 'store_nbr_44', 'store_nbr_45',
       'store_nbr_46', 'store_nbr_47', 'store_nbr_48', 'store_nbr_49',
       'store_nbr_5', 'store_nbr_50', 'store_nbr_51', 'store_nbr_52',
       'store_nbr_53', 'store_nbr_54', 'store_nbr_6', 'store_nbr_7',
       'store_nbr_8', 'store_nbr_9', 'region_Amazon', 'region_Coast',
       'region_Sierra', 'weekday_0', 'weekday_1', 'weekday_2', 'weekday_3',
       'weekday_4', 'weekday_5', 'weekday_6', 'month_1', 'month_10',
       'month_11', 'month_12', 'month_2', 'month_3', 'month_4', 'month_5',
       'month_6', 'month_7', 'month_8', 'month_9',
       't1', 't2', 't3', 't4', 't5', 't6', 't7']].values.astype(np.float32)) 

In [13]:
store_data_loader=DataLoader(dataset =X1, shuffle = False,batch_size=83110)

In [14]:
for i in store_data_loader:
    X_test=i

In [15]:
class FirstNetwork_v1(nn.Module):
    def __init__(self):
        super().__init__()
        torch.manual_seed(0)
        self.lin1 = nn.Linear(83,25)
        self.lin2 = nn.Linear(25, 1)
    
    def forward(self, X):
        a1 = self.lin1(X) #computes the dot product and adds bias
        h1 = a1.relu()
        a2 = self.lin2(h1) #computes dot product and adds bias
        return a2,a1

In [16]:
stores_model = FirstNetwork_v1()

In [17]:
stores_model.load_state_dict(torch.load("stores_model_final.pt"))
stores_model.eval()

FirstNetwork_v1(
  (lin1): Linear(in_features=83, out_features=25, bias=True)
  (lin2): Linear(in_features=25, out_features=1, bias=True)
)

In [18]:
output,store_features=stores_model(X_test)

In [19]:
store_data[["store_features_"+str(i) for i in range(1,26)]]=pd.DataFrame(store_features.data.numpy())

In [40]:
store_feature_combined=store_data[['date','store_nbr', 'store_features_1', 'store_features_2',
       'store_features_3', 'store_features_4', 'store_features_5',
       'store_features_6', 'store_features_7', 'store_features_8',
       'store_features_9', 'store_features_10', 'store_features_11',
       'store_features_12', 'store_features_13', 'store_features_14',
       'store_features_15', 'store_features_16', 'store_features_17',
       'store_features_18', 'store_features_19', 'store_features_20','store_features_21',
                                   'store_features_22','store_features_23','store_features_24','store_features_25']]

### Items_Model

In [21]:
m=[i for i in item_data.columns if "family" in i]

In [22]:
a=np.array(item_data[m])

In [23]:
item_data["family"]=[np.where(r==1)[0][0]+1 for r in a]

In [24]:
item_data["family"]=item_data.family.map(dict(zip(np.arange(1,34),[i[7:] for i in m])))

In [25]:
X1 = torch.tensor(item_data[['family_AUTOMOTIVE', 'family_BABY CARE', 'family_BEAUTY',
       'family_BEVERAGES', 'family_BOOKS', 'family_BREAD/BAKERY',
       'family_CELEBRATION', 'family_CLEANING', 'family_DAIRY', 'family_DELI',
       'family_EGGS', 'family_FROZEN FOODS', 'family_GROCERY I',
       'family_GROCERY II', 'family_HARDWARE', 'family_HOME AND KITCHEN I',
       'family_HOME AND KITCHEN II', 'family_HOME APPLIANCES',
       'family_HOME CARE', 'family_LADIESWEAR', 'family_LAWN AND GARDEN',
       'family_LINGERIE', 'family_LIQUOR,WINE,BEER', 'family_MAGAZINES',
       'family_MEATS', 'family_PERSONAL CARE', 'family_PET SUPPLIES',
       'family_PLAYERS AND ELECTRONICS', 'family_POULTRY',
       'family_PREPARED FOODS', 'family_PRODUCE',
       'family_SCHOOL AND OFFICE SUPPLIES', 'family_SEAFOOD', 'weekday_0',
       'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5',
       'weekday_6', 'month_1', 'month_10', 'month_11', 'month_12', 'month_2',
       'month_3', 'month_4', 'month_5', 'month_6', 'month_7', 'month_8',
       'month_9', 't1', 't2', 't3', 't4', 't5', 't6', 't7',]].values.astype(np.float32)) 

In [26]:
item_data_loader=DataLoader(dataset =X1, shuffle = False,batch_size=len(X1))

In [27]:
for i in item_data_loader:
    X_test=i

In [31]:
class SecondNetwork_v1(nn.Module):
    def __init__(self):
        super().__init__()
        torch.manual_seed(0)
        self.lin1 = nn.Linear(59,16)
        self.lin2 = nn.Linear(16, 1)
        
    
    def forward(self, X):
        a1 = self.lin1(X)
        h1 = a1.relu()
        a2 = self.lin2(h1)
        return a2,a1

In [32]:
items_model=SecondNetwork_v1()

In [33]:
items_model.load_state_dict(torch.load("items_model_final.pt"))
items_model.eval()

SecondNetwork_v1(
  (lin1): Linear(in_features=59, out_features=16, bias=True)
  (lin2): Linear(in_features=16, out_features=1, bias=True)
)

In [34]:
output,items_features=items_model(X_test)

In [38]:
item_data[["item_features_"+str(i) for i in range(1,17)]]=pd.DataFrame(items_features.data.numpy())

In [39]:
item_feature_combined=item_data[["date",'family', 'item_features_1', 'item_features_2',
       'item_features_3', 'item_features_4', 'item_features_5',
       'item_features_6', 'item_features_7', 'item_features_8',
       'item_features_9', 'item_features_10', 'item_features_11',
       'item_features_12', 'item_features_13', 'item_features_14',
       'item_features_15', 'item_features_16']]

### Final Model

In [41]:
final_data=data[["date","store_nbr","family","unit_sales"]].merge(store_feature_combined,on=["date","store_nbr"],
                                          how="left").merge(item_feature_combined,on=["date","family"],how="left")

In [42]:
final_data.dropna(axis=0,how="any",inplace=True)

In [43]:
final_data.drop(["date","store_nbr","family"],axis=1,inplace=True)

In [44]:
final_data.to_csv("final_data.csv",index=None)